<a href="https://colab.research.google.com/github/SaShakib/EEG_research/blob/main/EEG_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/Shakib(10sec_each)"

shakib_Backward_s10.json  shakib_Forward_s3.json  shakib_Left_s6.json	shakib_Rest_s9.json
shakib_Backward_s1.json   shakib_Forward_s4.json  shakib_Left_s7.json	shakib_right_s10.json
shakib_Backward_s2.json   shakib_Forward_s5.json  shakib_Left_s8.json	shakib_right_s1.json
shakib_Backward_s3.json   shakib_Forward_s6.json  shakib_Left_s9.json	shakib_right_s2.json
shakib_Backward_s4.json   shakib_Forward_s7.json  shakib_Rest_s10.json	shakib_right_s3.json
shakib_Backward_s5.json   shakib_Forward_s8.json  shakib_Rest_s1.json	shakib_right_s4.json
shakib_Backward_s6.json   shakib_Forward_s9.json  shakib_Rest_s2.json	shakib_right_s5.json
shakib_Backward_s7.json   shakib_Left_s10.json	  shakib_Rest_s3.json	shakib_right_s6.json
shakib_Backward_s8.json   shakib_Left_s1.json	  shakib_Rest_s4.json	shakib_right_s7.json
shakib_Backward_s9.json   shakib_Left_s2.json	  shakib_Rest_s5.json	shakib_right_s8.json
shakib_Forward_s10.json   shakib_Left_s3.json	  shakib_Rest_s6.json	shakib_right_s9.json
shaki

In [14]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Load and process raw data for LSTM
def load_raw_data(data_dir, fs=256, window_size=640, overlap=320):
    X, y = [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)  # Load the JSON file (list of objects)

            combined_data = []  # Accumulate data for aggregation
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    # Combine objects to form one window
                    segment = np.hstack(combined_data)[:, :window_size]  # [channels, window_size]
                    combined_data = []  # Reset for the next segment

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X.append(filtered_segment.T)  # Transpose to [window_size, channels]
                    y.append(filename.split("_")[1])  # Assign label based on filename
    return np.array(X), np.array(y)

# Load the data
data_dir = "/content/drive/My Drive/Shakib(10sec_each)"  # Directory containing your JSON files
X, y = load_raw_data(data_dir)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Build the LSTM Model


In [15]:
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.5),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(y_train.shape[1], activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the LSTM model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Evaluate LSTM
y_pred = model.predict(X_test)
predicted_classes = np.argmax(y_pred, axis=1)
true_classes = np.argmax(y_test, axis=1)

print(f"Classification Report:\n{classification_report(true_classes, predicted_classes, target_names=le.classes_)}")


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 718ms/step - accuracy: 0.1771 - loss: 1.6754 - val_accuracy: 0.2000 - val_loss: 1.6558
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 500ms/step - accuracy: 0.2938 - loss: 1.5990 - val_accuracy: 0.1750 - val_loss: 1.6632
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 501ms/step - accuracy: 0.3140 - loss: 1.5572 - val_accuracy: 0.1750 - val_loss: 1.6734
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 677ms/step - accuracy: 0.3074 - loss: 1.5402 - val_accuracy: 0.1500 - val_loss: 1.6861
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 774ms/step - accuracy: 0.2589 - loss: 1.5273 - val_accuracy: 0.1750 - val_loss: 1.6948
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 526ms/step - accuracy: 0.4348 - loss: 1.4546 - val_accuracy: 0.1750 - val_loss: 1.7038
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 497ms/step - accuracy: 0.4368 - loss: 1.4552 - val_accuracy: 0.1750 - val_loss: 1.7174
Epoch 8/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 933ms/step - accuracy: 0.5025 - loss: 1.3886 - val_accuracy: 0.2000 - val_loss: 1.7238
Epo

In [40]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter, welch
from scipy.stats import skew, kurtosis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Feature extraction function
def extract_features(window, fs):
    freqs, psd = welch(window, fs, nperseg=fs // 2)
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 40)}
    band_powers = {band: np.sum(psd[(freqs >= low) & (freqs < high)]) for band, (low, high) in bands.items()}
    stats = {'mean': np.mean(window), 'variance': np.var(window), 'skewness': skew(window), 'kurtosis': kurtosis(window)}
    return {**band_powers, **stats}

def load_data_for_rf_and_lstm(data_dir, fs=256, window_size=1024, overlap=512):
    X_rf, X_lstm, y = [], [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)  # Load the JSON file (list of objects)

            combined_data = []  # Accumulate data for aggregation
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    # Combine objects to form one window
                    segment = np.hstack(combined_data)[:, :window_size]  # [channels, window_size]
                    combined_data = []  # Reset for the next segment

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X_lstm.append(filtered_segment.T)  # For LSTM, shape: [window_size, channels]

                    # Flatten the segment for Random Forest
                    X_rf.append(filtered_segment.flatten())  # For RF, shape: [1, channels * window_size]

                    # Assign label
                    y.append(filename.split("_")[1])  # Label from filename
    return np.array(X_rf), np.array(X_lstm), np.array(y)

# Load the data
data_dir = "/content/drive/My Drive/Shakib(10sec_each)/Forward"  # Directory containing your JSON files
X_rf, X_lstm, y = load_data_for_rf_and_lstm(data_dir)

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split data into train and test sets
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(X_rf, y_encoded, test_size=0.2, random_state=42)
X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(X_lstm, y_cat, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_rf_train, y_rf_train)
y_rf_pred = rf.predict(X_rf_test)

print(f"Random Forest Accuracy: {accuracy_score(y_rf_test, y_rf_pred):.2f}")
print(classification_report(y_rf_test, y_rf_pred, target_names=le.classes_))



Random Forest Accuracy: 0.62
              precision    recall  f1-score   support

     Forward       0.67      0.50      0.57         4
        Rest       0.60      0.75      0.67         4

    accuracy                           0.62         8
   macro avg       0.63      0.62      0.62         8
weighted avg       0.63      0.62      0.62         8



In [35]:
model = Sequential([
    LSTM(100, input_shape=(X_lstm_train.shape[1], X_lstm_train.shape[2]), return_sequences=True),
    Dropout(0.5),
    LSTM(100, return_sequences=False),
    Dropout(0.5),
    Dense(y_lstm_train.shape[1], activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the LSTM model
model.fit(X_lstm_train, y_lstm_train, validation_data=(X_lstm_test, y_lstm_test), epochs=30, batch_size=128)

# Evaluate LSTM
y_lstm_pred = model.predict(X_lstm_test)
predicted_classes = np.argmax(y_lstm_pred, axis=1)
true_classes = np.argmax(y_lstm_test, axis=1)

print(f"Classification Report:\n{classification_report(true_classes, predicted_classes, target_names=le.classes_)}")


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.6734 - val_accuracy: 0.2500 - val_loss: 0.8003
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6562 - loss: 0.6054 - val_accuracy: 0.2500 - val_loss: 0.8075
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8125 - loss: 0.5672 - val_accuracy: 0.3750 - val_loss: 0.8244
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 0.5429 - val_accuracy: 0.3750 - val_loss: 0.8562
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 0.5023 - val_accuracy: 0.3750 - val_loss: 0.9081
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8438 - loss: 0.4360 - val_accuracy: 0.3750 - val_loss: 0.9718
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.8750 - loss: 0.4068 - val_accuracy: 0.3750 - val_loss: 1.0395
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9688 - loss: 0.3337 - val_accuracy: 0.3750 - val_loss: 1.1165
Epoch 9/30
1/1 ━━━━━━━━━━━━

In [20]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Load and preprocess raw EEG data
def load_eeg_data(data_dir, fs=256, window_size=256, overlap=128):
    X, y = [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)  # Load the JSON file (list of objects)

            combined_data = []  # Accumulate data for aggregation
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    # Combine objects to form one window
                    segment = np.hstack(combined_data)[:, :window_size]  # [channels, window_size]
                    combined_data = []  # Reset for the next segment

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X.append(filtered_segment.T)  # Transpose to [time_steps, channels]
                    y.append(filename.split("_")[1])  # Assign label based on filename
    return np.array(X), np.array(y)

# Load the data
data_dir = "/content/drive/My Drive/Shakib(10sec_each)"  # Directory containing your JSON files
X, y = load_eeg_data(data_dir)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout

# Build the CNN-LSTM Model
def build_cnn_lstm(input_shape, num_classes):
    model = Sequential([
        # CNN layers
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # LSTM layers
        LSTM(64, return_sequences=False),
        Dropout(0.5),

        # Dense output layer
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define model input shape and number of classes
input_shape = (X_train.shape[1], X_train.shape[2])  # (time_steps, channels)
num_classes = y_train.shape[1]

# Build the model
model = build_cnn_lstm(input_shape, num_classes)

# Train the CNN-LSTM model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 169ms/step - accuracy: 0.1593 - loss: 1.9134 - val_accuracy: 0.2000 - val_loss: 1.6591
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.2182 - loss: 1.7479 - val_accuracy: 0.1500 - val_loss: 1.6721
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.1531 - loss: 1.7533 - val_accuracy: 0.1400 - val_loss: 1.6862
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.2314 - loss: 1.7153 - val_accuracy: 0.1900 - val_loss: 1.6800
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.2400 - loss: 1.6438 - val_accuracy: 0.1400 - val_loss: 1.6534
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.2054 - loss: 1.6669 - val_accuracy: 0.1300 - val_loss: 1.6657
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.2005 - loss: 1.6771 - val_accuracy: 0.1700 - val_loss: 1.6374
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.2397 - loss: 1.6282 - val_accuracy: 0.2200 

In [26]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout

# Build LRCN model
def build_lrcn(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # CNN feature extractor
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    # LSTM sequence modeler
    x = LSTM(64, return_sequences=False)(x)
    x = Dropout(0.5)(x)

    # Output layer
    output = Dense(num_classes, activation='softmax')(x)

    # Build model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Ensure input shape is correct: (time_steps, features)
input_shape_lrcn = (X_train.shape[1], X_train.shape[2])  # Shape: (time_steps, channels)

# Build and train the LRCN model
model_lrcn = build_lrcn(input_shape_lrcn, y_train.shape[1])
model_lrcn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Evaluate LRCN
loss_lrcn, accuracy_lrcn = model_lrcn.evaluate(X_test, y_test, verbose=0)
print(f"LRCN Test Accuracy: {accuracy_lrcn:.2f}")


Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - accuracy: 0.1727 - loss: 1.8969 - val_accuracy: 0.1900 - val_loss: 1.6707
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.2126 - loss: 1.7449 - val_accuracy: 0.1600 - val_loss: 1.6968
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step - accuracy: 0.2589 - loss: 1.6113 - val_accuracy: 0.1800 - val_loss: 1.6394
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.2152 - loss: 1.7396 - val_accuracy: 0.2700 - val_loss: 1.6310
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.1832 - loss: 1.7306 - val_accuracy: 0.2100 - val_loss: 1.6385
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.2126 - loss: 1.6724 - val_accuracy: 0.2200 - val_loss: 1.6495
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.2656 - loss: 1.6490 - val_accuracy: 0.1300 - val_loss: 1.6800
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.1938 - loss: 1.6859 - val_accuracy: 0.1900 -

In [30]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense, Dropout

# Reshape data for ConvLSTM (add spatial dimensions and channel dimension)
def reshape_for_convlstm(X):
    return X[:, :, np.newaxis, :, np.newaxis]  # Shape: (samples, time_steps, 1, channels, 1)

# Build ConvLSTM model
def build_convlstm(input_shape, num_classes):
    model = Sequential([
        ConvLSTM2D(filters=64, kernel_size=(1, 2), activation='relu', input_shape=input_shape, return_sequences=True),
        BatchNormalization(),
        ConvLSTM2D(filters=64, kernel_size=(1, 2), activation='relu', return_sequences=False),
        BatchNormalization(),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Reshape for ConvLSTM
X_train_clstm = reshape_for_convlstm(X_train)  # Shape: (samples, time_steps, 1, channels, 1)
X_test_clstm = reshape_for_convlstm(X_test)
input_shape_clstm = X_train_clstm.shape[1:]  # Input shape excluding batch size

# Build and train the ConvLSTM model
model_clstm = build_convlstm(input_shape_clstm, y_train.shape[1])
model_clstm.fit(X_train_clstm, y_train, validation_data=(X_test_clstm, y_test), epochs=10, batch_size=32)

# Evaluate ConvLSTM
loss_clstm, accuracy_clstm = model_clstm.evaluate(X_test_clstm, y_test, verbose=0)
print(f"ConvLSTM Test Accuracy: {accuracy_clstm:.2f}")


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.1803 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.1810 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.1951 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.2181 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.2029 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.1965 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.1902 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.1920 - loss: nan - val_accuracy: 0.2200 - val_loss: nan
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 41